# 自訂列舉

要讓一群值映射到另一群值(有任意字元)

## 方法一：使用唯獨屬性的靜態Class
不建議，如果有其他接手的人在 Class 加了其他方法，就失去了這個 Class 的原意了

In [ ]:
using System.ComponentModel;

public class DescID : DescriptionAttribute {
	public DescID(string text) : base(text) {
	}
}

public class Seasons {
	public static string Spring { get { return "-春天-"; } }
	public static string Summer { get { return "-夏天-"; } }
	public static string Autumn { get { return "-秋天-"; } }
	public static string Winter { get { return "-冬天-"; } }

	// public void Print() => Console.WriteLine("我是 Seasons 物件");
	// public void Show() => Console.WriteLine("我是 Seasons 物件");
	// public void Log() => Console.WriteLine("我是 Seasons 物件");
}

string descSummer = Seasons.Summer;
descSummer

## 方法二：利用 Description Attribute 加反射

In [ ]:
using System.Runtime.Serialization;
using System.ComponentModel;
using System.Reflection;

public static string GetDescription<T>(this T pObj, Type? pType = null) {
	if (pObj is null)
		return string.Empty;

	Type type = pObj.GetType();
	FieldInfo? fi = type.GetField(pObj.ToString() ?? string.Empty);

	if (fi is null)
		return string.Empty;

	Type targetType = pType ?? typeof(DescriptionAttribute);
	var attributes = fi.GetCustomAttributes(targetType, false);
	if (attributes.Length == 0)
		return string.Empty;

	object? attribute;
	attribute = attributes.FirstOrDefault(x => x.GetType() == targetType);
	if (attribute is null)
		return string.Empty;

	var targetAttribute = Convert.ChangeType(attribute, targetType);
	if (targetAttribute.PropertyExists("Description"))
		return ((dynamic)targetAttribute).Description;

	return string.Empty;
}

public static bool PropertyExists<T>(this T pObj, string pPropertyName) {
	if (pObj is null)
		return false;

	if (pObj is IDictionary<string, object> dict) {
		return dict.ContainsKey(pPropertyName);
	}

	return pObj.GetType().GetProperty(pPropertyName) != null;
}

public class DescID : DescriptionAttribute {
	public DescID(string text) : base(text) {
	}
}
public class DescIP : DescriptionAttribute {
	public DescIP(string text) : base(text) {
	}
}

public enum OpCustomer {
	[Description("- Description A -")]
	[DescID("000000000000")]
	[DescIP("0.0.0.0")]
	A股份有限公司,

	[Description("- Description B -")]
	[DescID("111111111111")]
	[DescIP("1.1.1.1")]
	B股份有限公司
}

var strDesc = GetDescription(OpCustomer.A股份有限公司);
Console.WriteLine(strDesc);
strDesc = OpCustomer.A股份有限公司.GetDescription(typeof(DescID));
Console.WriteLine(strDesc);
strDesc = OpCustomer.A股份有限公司.GetDescription(typeof(DescIP));
Console.WriteLine(strDesc);